# Skipgrams over Skipgrams

## Setup

In [1]:
# addprocs([("chfin-tp", :auto)])
# addprocs()

In [2]:
using DigitalMusicology

In [3]:
@everywhere DigitalMusicology.usekern("/home/chfin/Uni/phd/data/csapp/mozart-piano-sonatas/");

In [4]:
# contains the "skipgrams over skipgrams" code
@everywhere include("skipgrams.jl");
using Unsims

## Analyzing a piece

In [5]:
prinnerpiece = "sonata03-2"
notes = getpiece(prinnerpiece, :notes_secs);

The notes are grouped to non-simultaneous note groups ("unsims")

In [6]:
mozunsims = unsims(notes, 1.0, 2, 1.0);

Unsims are again grouped to skipgrams.
Output skipgrams are sampled with `p = 0.00001`.

In [7]:
sgs = unsimskipgrams(mozunsims, 1.0, 4, 0.00001);

The resulting SG^2s are reduced to their pitch content (notes -> pitches),
each group is sorted by ascending pitch,
pitches are trasposed relative to the first pitch in the first group (reference pitch)
and converted to pitch classes (`schemacandify`).
This allows to reproduce, which pc appeared in which voice, relative to the reference note.

The resulting "schema candidates" are counted (with equal weights for now):

In [8]:
counts = countmapby(sgs, schemacandify);

The counted schema candidates are ranked by their counts.

In [9]:
ranks = rankcounts(counts)
topranks(ranks, 100);

4.0: [0, 3] -> [11, 10] -> [5, 0] -> [7, 8]
4.0: [0, 3] -> [10, 3] -> [4, 5] -> [0, 0]
3.0: [0, 1] -> [1, 5] -> [8, 5] -> [8, 1]
3.0: [0, 1] -> [3, 5] -> [10, 3] -> [8, 10]
3.0: [0, 8] -> [0, 0] -> [1, 5] -> [3, 8]
3.0: [0, 8] -> [1, 5] -> [3, 0] -> [8, 7]
3.0: [0, 8] -> [0, 8] -> [8, 8] -> [1, 8]
3.0: [0, 8] -> [1, 7] -> [0, 5] -> [3, 10]
3.0: [0, 8] -> [10, 6] -> [10, 3] -> [0, 5]
3.0: [0, 3] -> [0, 1] -> [0, 0] -> [3, 8]
3.0: [0, 3] -> [10, 5] -> [3, 3] -> [7, 10]
3.0: [0, 3] -> [3, 8] -> [10, 3] -> [3, 10]
3.0: [0, 3] -> [2, 2] -> [3, 10] -> [3, 3]
3.0: [0, 3] -> [1, 7] -> [3, 5] -> [3, 7]
3.0: [0, 3] -> [1, 7] -> [5, 0] -> [3, 8]
3.0: [0, 3] -> [6, 8] -> [11, 11] -> [3, 11]
3.0: [0, 6] -> [8, 1] -> [8, 10] -> [5, 1]
3.0: [0, 6] -> [6, 10] -> [8, 10] -> [6, 10]
3.0: [0, 6] -> [3, 7] -> [10, 1] -> [0, 3]
3.0: [0, 5] -> [9, 5] -> [9, 0] -> [5, 5]
3.0: [0, 5] -> [4, 0] -> [7, 4] -> [0, 11]
3.0: [0, 5] -> [5, 10] -> [5, 0] -> [0, 5]
3.0: [0, 5] -> [10, 10] -> [5, 9] -> [5, 9]
3.0: [0, 

As the schema candidates are dominated by artifacts (trills?), it is necessary to filter out the problematic cases.

In [10]:
# trill
istrill(cand) =
    all(stage -> all(p -> p==midi(2) || p==midi(0), stage), cand) ||
    all(stage -> all(p -> p==midi(10) || p==midi(0), stage), cand)

notrills = filter(r -> !istrill(r[1]), ranks)
topranks(notrills, 20)

4.0: [0, 3] -> [11, 10] -> [5, 0] -> [7, 8]
4.0: [0, 3] -> [10, 3] -> [4, 5] -> [0, 0]
3.0: [0, 1] -> [1, 5] -> [8, 5] -> [8, 1]
3.0: [0, 1] -> [3, 5] -> [10, 3] -> [8, 10]
3.0: [0, 8] -> [0, 0] -> [1, 5] -> [3, 8]
3.0: [0, 8] -> [1, 5] -> [3, 0] -> [8, 7]
3.0: [0, 8] -> [0, 8] -> [8, 8] -> [1, 8]
3.0: [0, 8] -> [1, 7] -> [0, 5] -> [3, 10]
3.0: [0, 8] -> [10, 6] -> [10, 3] -> [0, 5]
3.0: [0, 3] -> [0, 1] -> [0, 0] -> [3, 8]
3.0: [0, 3] -> [10, 5] -> [3, 3] -> [7, 10]
3.0: [0, 3] -> [3, 8] -> [10, 3] -> [3, 10]
3.0: [0, 3] -> [2, 2] -> [3, 10] -> [3, 3]
3.0: [0, 3] -> [1, 7] -> [3, 5] -> [3, 7]
3.0: [0, 3] -> [1, 7] -> [5, 0] -> [3, 8]
3.0: [0, 3] -> [6, 8] -> [11, 11] -> [3, 11]
3.0: [0, 6] -> [8, 1] -> [8, 10] -> [5, 1]
3.0: [0, 6] -> [6, 10] -> [8, 10] -> [6, 10]
3.0: [0, 6] -> [3, 7] -> [10, 1] -> [0, 3]
3.0: [0, 5] -> [9, 5] -> [9, 0] -> [5, 5]


In [11]:
# trill part
istrillpart(cand) =
    any(midis(0:11)) do trans
        count(stage -> all(p -> pc(p+trans) ∈ [midi(0), midi(2)], stage), cand) > 1
    end

notrillparts = filter(r -> !istrillpart(r[1]), notrills)
topranks(notrillparts, 20)

4.0: [0, 3] -> [11, 10] -> [5, 0] -> [7, 8]
4.0: [0, 3] -> [10, 3] -> [4, 5] -> [0, 0]
3.0: [0, 1] -> [1, 5] -> [8, 5] -> [8, 1]
3.0: [0, 1] -> [3, 5] -> [10, 3] -> [8, 10]
3.0: [0, 8] -> [0, 0] -> [1, 5] -> [3, 8]
3.0: [0, 8] -> [1, 5] -> [3, 0] -> [8, 7]
3.0: [0, 8] -> [0, 8] -> [8, 8] -> [1, 8]
3.0: [0, 8] -> [1, 7] -> [0, 5] -> [3, 10]
3.0: [0, 8] -> [10, 6] -> [10, 3] -> [0, 5]
3.0: [0, 3] -> [0, 1] -> [0, 0] -> [3, 8]
3.0: [0, 3] -> [10, 5] -> [3, 3] -> [7, 10]
3.0: [0, 3] -> [3, 8] -> [10, 3] -> [3, 10]
3.0: [0, 3] -> [2, 2] -> [3, 10] -> [3, 3]
3.0: [0, 3] -> [1, 7] -> [3, 5] -> [3, 7]
3.0: [0, 3] -> [1, 7] -> [5, 0] -> [3, 8]
3.0: [0, 3] -> [6, 8] -> [11, 11] -> [3, 11]
3.0: [0, 6] -> [8, 1] -> [8, 10] -> [5, 1]
3.0: [0, 6] -> [6, 10] -> [8, 10] -> [6, 10]
3.0: [0, 6] -> [3, 7] -> [10, 1] -> [0, 3]
3.0: [0, 5] -> [9, 5] -> [9, 0] -> [5, 5]


In [12]:
fst = findfirstusg(mozunsims, map(midis, [[0,4], [0,4], [0,5], [0,0]]), 1.0, 4, 1)

LoadError: [91mUndefVarError: findfirstusg not defined[39m

In [13]:
schemacandify(fst)

LoadError: [91mUndefVarError: fst not defined[39m